# 교차 검증, 하이퍼파라미터 튜닝
- 가장 좋은 모델 선정

In [32]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np   # 수치 계산을 위한 numpy
from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트 분류기
from sklearn.svm import SVC  # 서포트 벡터 머신
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.metrics import accuracy_score  # 정확도 평가 지표

# 데이터 가져오기
data = pd.read_csv('train.csv')
data.head()

# 모델링에 사용할 특성(feature)과 타겟(target) 변수 선택
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']  # 승객 등급, 성별, 나이, 동반자 수, 요금, 탑승 항구
target = 'Survived'  # 생존 여부 (0: 사망, 1: 생존)
data['Survived'].value_counts()  # 0과 1의 비율이 5:5는 아님 => 클래스 불균형(Class Imbalanced)

Survived
0    549
1    342
Name: count, dtype: int64

In [33]:
# 결측치 확인
print(data.isnull().sum())

data.info()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# 데이터 전처리
- 고려사항
  + 전처리를 전체 데이터에 해도 무방한가? 아니면 훈련데이터에만 적용해야 하는가?
  + 범주형 데이터는 2가지 선택지가 존재 : 1) One-Hot Encoder (= 동일 가중치 부여) 2) Label Encoder (0, 1, 2)
  + Label Encoder 사용하는 경우 => 탐색적 데이터 분석 & 사회적인 통념을 고려해서 전처리
    - ex) 타이타닉 여성 생존자가 남성보다 많음 (레이디 퍼스트 문화) => 아래처럼 여성=1로 여성에게 더 높은 가중치 부여

In [34]:
# data['Sex'].unique()
data['Sex'] = data['Sex'].map({'male' : 0, 'female' : 1})  # 남성=0, 여성=1
data['Embarked'] = data['Embarked'].map({'C' : 0, 'Q' : 1, 'S' : 2})  # 타이타닉 C 칸에서 사망자 많음 => 가중치 0

In [35]:
data['Embarked'].value_counts()

Embarked
2.0    644
0.0    168
1.0     77
Name: count, dtype: int64

# 데이터셋 분리

In [40]:
# 아래의 features에서 고려하지 않은 feature(= 학습에 이용하지 않는 독립변수) 3개 : ID, Name, Cabin 
# ID, Name 제거 이유 : 패턴이 발견되지 않는 변수는 제외
# Cabin 제거 이유 : 결측치의 개수가 너무 많아서 제외
# 참고 자료 : https://scikit-learn.org/stable/modules/impute.html
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']  # 승객 등급, 성별, 나이, 동반자 수, 요금, 탑승 항구
target = 'Survived' 

X = data[features]  # 독립 변수
y = data[target]    # 종속 변수

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, stratify=y    # test_size default값 : 75/25
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((668, 7), (223, 7), (668,), (223,))

# Feature Engineering

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 668 entries, 486 to 821
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    668 non-null    int64  
 1   Sex       668 non-null    int64  
 2   Age       537 non-null    float64
 3   SibSp     668 non-null    int64  
 4   Parch     668 non-null    int64  
 5   Fare      668 non-null    float64
 6   Embarked  666 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 41.8 KB


In [45]:
# 훈련데이터의 정보를 이용해서 테스트 테이터에 적용
# 훈련데이터에서 결측치를 채울 대표값 계산
age_mean = round(X_train['Age'].mean(), 0)     # 승객 나이의 평균값        
embarked_mode = X_train['Embarked'].mode()[0]  # 탑승 항구의 최빈값
fare_mean = X_train['Fare'].mean()             # 요금의 평균값
pclass_mode = X_train['Pclass'].mode()[0]      # 승객 등급의 최빈값


print(age_mean, embarked_mode)

# 훈련데이터 적용
X_train['Age'] = X_train['Age'].fillna(age_mean)
X_train['Embarked'] = X_train['Embarked'].fillna(embarked_mode)

# 테스트 데이터 적용
X_test['Age'] = X_test['Age'].fillna(age_mean)
X_test['Embarked'] = X_test['Embarked'].fillna(embarked_mode)

30.0 2.0


- 정규화, 표준화, One-Hot Encoding 고려해야 하나, 여기서는 안함
  + 이유 : 연속형 데이터로 볼만한 것이 없음 => 대부분이 이산형 데이터(Count 데이터)

# 모델링
- 각 모델 별 하이퍼파라미터 후보군 정의
- 교차 검증
- 가장 최고의 모델을 선정
  + 여러 모델 참고 자료 : https://scikit-learn.org/stable/modules/ensemble.html

- Decision Tree는 부등호로 구분
  + Gini, Entropy로 독립변수 구분
- RandomForest : Decision Tree의 확장판
  + tree1 : 사망
  + tree2 : 생존
  + tree3 : 사망
  + 결론 : 사망2/생존1 이니까, 이 모델은 사망으로 판단

In [49]:
# 사용할 모델 정의
models = {
    'RandomForest': RandomForestClassifier(random_state=42),   # 랜덤 포레스트 분류기
    'SVM': SVC(random_state=42, probability=True),             # 서포트 벡터 머신 (선형 모델 but, 현재는 퍼셉트론 모델을 주로 사용)
    'LogisticRegression': LogisticRegression(random_state=42)  # 로지스틱 회귀
}

# 각 모델별 하이퍼파라미터 후보군 정의
param_grid = {
    'RandomForest': [
        {'n_estimators': 100, 'max_depth': None},  # 트리 100개, 깊이 제한 없음
        {'n_estimators': 200, 'max_depth': 5}      # 트리 200개, 최대 깊이 5
    ],
    'SVM': [
        {'C': 1.0, 'kernel': 'rbf'},    # RBF 커널, C=1.0
        {'C': 0.5, 'kernel': 'linear'}  # 선형 커널, C=0.5
    ],
    'LogisticRegression': [
        {'C': 1.0, 'max_iter': 1000},  # 기본 설정
        {'C': 0.1, 'max_iter': 1000}   # 더 강한 정규화
    ]
}

# 교차 검증을 통한 최적 모델 선정
best_score = 0          # 최고 성능 점수
best_model_name = None  # 최고 성능 모델 이름
best_model = None       # 최고 성능 모델 객체


# 각 모델과 하이퍼파라미터 조합에 대해 교차 검증 수행
for model_name, model in models.items():
    print(f"\n--- Testing {model_name} ---")
    for params in param_grid[model_name]:
        model.set_params(**params)  # 하이퍼파라미터 설정
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')  # 5-fold 교차검증
        mean_cv = np.mean(cv_scores)  # 평균 교차검증 점수
        print(f"Params: {params}, CV Accuracy: {mean_cv:.4f}")
        
        # 최고 성능 모델 업데이트
        if mean_cv > best_score:
            best_score = mean_cv
            best_model_name = model_name
            best_model = model.set_params(**params)

# 최종 선택된 모델로 테스트셋 평가
best_model.fit(X_train, y_train)  # 최적 모델 학습
y_pred = best_model.predict(X_test)  # 테스트셋 예측
test_acc = accuracy_score(y_test, y_pred)  # 테스트셋 정확도 계산

# 최종 결과 출력
print(f"\nBest Model: {best_model_name}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Test Set Accuracy: {test_acc:.4f}")


--- Testing RandomForest ---
Params: {'n_estimators': 100, 'max_depth': None}, CV Accuracy: 0.8249
Params: {'n_estimators': 200, 'max_depth': 5}, CV Accuracy: 0.8264

--- Testing SVM ---
Params: {'C': 1.0, 'kernel': 'rbf'}, CV Accuracy: 0.6931
Params: {'C': 0.5, 'kernel': 'linear'}, CV Accuracy: 0.7934

--- Testing LogisticRegression ---
Params: {'C': 1.0, 'max_iter': 1000}, CV Accuracy: 0.8024
Params: {'C': 0.1, 'max_iter': 1000}, CV Accuracy: 0.8084

Best Model: RandomForest
Best CV Score: 0.8264
Test Set Accuracy: 0.7758


In [51]:
# 교차 검증에 걸린 시간까지 계산한 코드
import time
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# 사용할 모델 정의
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'LogisticRegression': LogisticRegression(random_state=42)
}

# 각 모델별 하이퍼파라미터 후보군 정의
param_grid = {
    'RandomForest': [
        {'n_estimators': 100, 'max_depth': None},
        {'n_estimators': 200, 'max_depth': 5}
    ],
    'SVM': [
        {'C': 1.0, 'kernel': 'rbf'},
        {'C': 0.5, 'kernel': 'linear'}
    ],
    'LogisticRegression': [
        {'C': 1.0, 'max_iter': 1000},
        {'C': 0.1, 'max_iter': 1000}
    ]
}

# 결과 저장 변수 초기화
best_score = 0
best_model_name = None
best_model = None
best_cv_time = 0  # 최적 모델의 CV 수행 시간 저장용

# 각 모델과 하이퍼파라미터 조합에 대해 교차 검증 수행
for model_name, model in models.items():
    print(f"\n--- Testing {model_name} ---")
    
    for params in param_grid[model_name]:
        model.set_params(**params)
        
        start_time = time.time()
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
        end_time = time.time()
        
        mean_cv = np.mean(cv_scores)
        elapsed = end_time - start_time
        
        print(f"Params: {params}, CV Accuracy: {mean_cv:.4f}, Time: {elapsed:.2f} sec")
        
        if mean_cv > best_score:
            best_score = mean_cv
            best_model_name = model_name
            best_model = model.set_params(**params)
            best_cv_time = elapsed  # 여기서 최적 모델의 CV 시간 저장

# 최종 선택된 모델로 테스트셋 평가
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)

# 최종 결과 출력
print(f"\nBest Model: {best_model_name}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Test Set Accuracy: {test_acc:.4f}")
print(f"Best Model Cross-Validation Time: {best_cv_time:.2f} sec")


--- Testing RandomForest ---
Params: {'n_estimators': 100, 'max_depth': None}, CV Accuracy: 0.8249, Time: 0.68 sec
Params: {'n_estimators': 200, 'max_depth': 5}, CV Accuracy: 0.8264, Time: 1.20 sec

--- Testing SVM ---
Params: {'C': 1.0, 'kernel': 'rbf'}, CV Accuracy: 0.6931, Time: 0.24 sec
Params: {'C': 0.5, 'kernel': 'linear'}, CV Accuracy: 0.7934, Time: 76.87 sec

--- Testing LogisticRegression ---
Params: {'C': 1.0, 'max_iter': 1000}, CV Accuracy: 0.8024, Time: 0.11 sec
Params: {'C': 0.1, 'max_iter': 1000}, CV Accuracy: 0.8084, Time: 0.11 sec

Best Model: RandomForest
Best CV Score: 0.8264
Test Set Accuracy: 0.7758
Best Model Cross-Validation Time: 1.20 sec


# 전처리 정보 저장 & 모델 저장해서 내보내기

In [53]:
import joblib   # 모델 저장을 위한 라이브러리
import json     # 전처리 정보 저장을 위한 json

# 전처리 정보 저장
preprocessing_info = {
    'age_mean': float(age_mean),
    'fare_mean': float(fare_mean),
    'pclass_mode': int(pclass_mode),
    'embarked_mode': int(embarked_mode),
    'features': features,
    'sex_mapping': {'male': 0, 'female': 1},
    'embarked_mapping': {'C': 0, 'Q': 1, 'S': 2}
}

# 모델과 전처리 정보 저장
joblib.dump(best_model, 'titanic_model.joblib')

# 훈련 데이터의 정보를 추가
with open('preprocessing_info.json', 'w') as f:
    json.dump(preprocessing_info, f)

# 최종 결과 출력
print(f"\nBest Model: {best_model_name}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Test Set Accuracy: {test_acc:.4f}")
print("\nModel and preprocessing information have been saved.")


Best Model: RandomForest
Best CV Score: 0.8264
Test Set Accuracy: 0.7758

Model and preprocessing information have been saved.


- 전체 모델 만들기 복습 및 Predict Calorie Expenditure(CH03의 데이터) 캐글 대회 데이터에도 적용
  + 코드 추가할 것 : 교차 검증 시, 시간 측정 하기
  + 하이퍼파라미터 튜닝 개수를 늘릴 때마다 시간이 제법 많이 소요됨을 확인해봐라